<a href="https://colab.research.google.com/github/aayushJkumar/Auto_Img_Bg_Remover_MODNet/blob/main/Auto_Image_Remover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
from PIL import Image
import cv2
import os
import numpy as np
import pandas as pd
from google.colab import files
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# clone the repository
%cd /content
if not os.path.exists('MODNet'):
  !git clone https://github.com/ZHKKKe/MODNet
%cd MODNet/

# dowload the pre-trained ckpt for image matting
pretrained_ckpt = 'pretrained/modnet_photographic_portrait_matting.ckpt'
if not os.path.exists(pretrained_ckpt):
  !gdown --id 1mcr7ALciuAsHCpLnrtG_eop5-EYhbCmz \
          -O pretrained/modnet_photographic_portrait_matting.ckpt

/content
/content/MODNet


In [ ]:
# clean and rebuild the image folders
input_folder = 'demo/image_matting/colab/input'
if os.path.exists(input_folder):
  shutil.rmtree(input_folder)
os.makedirs(input_folder)

output_folder = 'demo/image_matting/colab/output'
if os.path.exists(output_folder):
  shutil.rmtree(output_folder)
os.makedirs(output_folder)
image_names = os.listdir('/content/drive/MyDrive/IIT GUWAHATI Internship Task TASK/Frontal Images')
for i in image_names :
  shutil.move(os.path.join('/content/drive/MyDrive/IIT GUWAHATI Internship Task TASK/Frontal Images',i), os.path.join(input_folder,i))

In [ ]:
!python -m demo.image_matting.colab.inference \
        --input-path demo/image_matting/colab/input \
        --output-path demo/image_matting/colab/output \
        --ckpt-path ./pretrained/modnet_photographic_portrait_matting.ckpt

Process image: 000073.jpg
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3680: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  "The default behavior for interpolate/upsample with float scale_factor changed "
Process image: 000003.jpg
Process image: 000010.jpg
Process image: 000088.jpg
Process image: 000072.jpg
Process image: 000052.jpg
Process image: 000029.jpg
Process image: 000050.jpg
Process image: 000076.jpg
Process image: 000023.jpg
Process image: 000078.jpg
Process image: 000046.jpg
Process image: 000063.jpg
Process image: 000004.jpg
Process image: 000082.jpg
Process image: 000091.jpg
Process image: 000031.jpg
Process image: 000069.jpg
Process image: 000045.j

In [ ]:
import numpy as np
from PIL import Image
c=1
def combined_display(image, matte,name):
  # calculate display resolution
  w, h = image.width, image.height
  rw, rh = 800, int(h * 800 / (3 * w))
  
  # obtain predicted foreground
  image = np.asarray(image)
  if len(image.shape) == 2:
    image = image[:, :, None]
  if image.shape[2] == 1:
    image = np.repeat(image, 3, axis=2)
  elif image.shape[2] == 4:
    image = image[:, :, 0:3]
  matte = np.repeat(np.asarray(matte)[:, :, None], 3, axis=2) / 255
  foreground = image * matte + np.full(image.shape, 255) * (1 - matte)
  
  # combine image, foreground, and alpha into one line
  # combined = np.concatenate((image, foreground, matte * 255), axis=1)
  # combined = Image.fromarray(np.uint8(combined)).resize((rw, rh))
  # combined = Image.fromarray(np.uint8(foreground)).resize((rw, rh))
  combined = Image.fromarray(np.uint8(foreground)).resize((w,h))
  # s=name
  # combined.save(os.path.join('/content/drive/MyDrive/IIT GUWAHATI Internship Task TASK/Frontal_Images_output',name))
  
  return combined

# visualize all images
image_names = os.listdir(input_folder)
for image_name in image_names:
  matte_name = image_name.split('.')[0] + '.png'
  image = Image.open(os.path.join(input_folder, image_name))
  matte = Image.open(os.path.join(output_folder, matte_name))
  display(combined_display(image, matte,image_name))
  combined_display(image, matte,image_name).save('/content/drive/MyDrive/IIT GUWAHATI Internship Task TASK/IMg_Bg_Removed'+'/'+ image_name)
  print(image_name, '\n')

In [ ]:
# zip_filename = 'matte.zip'
# if os.path.exists(zip_filename):
#   os.remove(zip_filename)

# os.system(f"zip -r -j {zip_filename} {output_folder}/*")
# files.download(zip_filename)

In [ ]:
# image_names = os.listdir(input_folder)
# for i in image_names :
#     cur_image_path = os.path.join(input_folder,i)
#     cur_image_out_path = os.path.join('/content/drive/MyDrive/IIT GUWAHATI Internship Task TASK/Frontal_Images_output',i)
#     shutil.copyfile(cur_image_path, cur_image_out_path)